___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Кросс-валидация - введение

В этой серии лекций мы подробно рассмотрим различные методы кросс-валидации, а также обсудим общую идею кросс-валидации. Хорошую документацию по этой теме можно почитать вот здесь: https://scikit-learn.org/stable/modules/cross_validation.html

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Пример данных

In [3]:
df = pd.read_csv("../DATA/Advertising.csv")

In [4]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Разбиваем данные на обучающий и тестовый наборы данных - Train | Test Split

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Применяем масштабирование (scale) для тестовых данных X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на тестовых данных X (создавая предсказания и сравнивая их с Y_test)
7. Уточняем параметры модели, повторяя шаги 5 и 6

In [5]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабирование данных - SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Создаём модель**

In [2]:
from sklearn.linear_model import Ridge

In [7]:
# Мы осознанно указываем неудачное значение Alpha!
model = Ridge(alpha=100)

In [8]:
model.fit(X_train,y_train)

Ridge(alpha=100, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [9]:
y_pred = model.predict(X_test)

**Оценка модели (evaluation)**

In [10]:
from sklearn.metrics import mean_squared_error

In [11]:
mean_squared_error(y_test,y_pred)

7.34177578903413

**Уточняем параметры и заново оцениваем модель**

In [12]:
model = Ridge(alpha=1)

In [13]:
model.fit(X_train,y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [14]:
y_pred = model.predict(X_test)

**Повторная оценка**

In [15]:
mean_squared_error(y_test,y_pred)

2.319021579428752

Намного лучше! Мы можем повторять эти шаги до тех пор, пока не получим хорошие метрики производительности. (Ранее мы видели, как для этой цели использовать RidgeCV, но здесь мы можем обобщить процесс кросс-валидации для случая любой модели).

----
----
----
## Обучение - валидация - тестирование (Train | Validation | Test Split Procedure)

Этот подход ещё называют методом с задержкой ("hold-out"), потому что здесь мы не настраиваем параметры модели на финальном тестовом наборе данных, а используем тестовые данные *только* для оценки финального результата.

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий, оценочный и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Масштабируем (scale) оценочные данные X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на оценочных данных X (создавая предсказания и сравнивая их с Y_eval)
7. Уточняем параметры модели, повторяя шаги 5 и 6
8. Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)

In [16]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

In [17]:
###############################################################################################
#### Вызываем SPLIT дважды! Здесь мы создаём три набора данных - TRAIN, VALIDATION и TEST  #########
###############################################################################################
from sklearn.model_selection import train_test_split

# 70% данных определяем в обучающий набор, остальные 30% откладываем в сторону
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Оставшиеся 30% разбиваем на оценочный и тестовый наборы данных
# Каждый будет по 15% от исходного набора данных 
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [18]:
# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Создаём модель**

In [19]:
from sklearn.linear_model import Ridge

In [20]:
# Мы осознанно указываем неудачное значение Alpha!
model = Ridge(alpha=100)

In [21]:
model.fit(X_train,y_train)

Ridge(alpha=100, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [22]:
y_eval_pred = model.predict(X_eval)

**Оценка модели (Evaluation)**

In [23]:
from sklearn.metrics import mean_squared_error

In [24]:
mean_squared_error(y_eval,y_eval_pred)

7.320101458823871

**Уточняем параметры и повторно оцениваем модель**

In [25]:
model = Ridge(alpha=1)

In [26]:
model.fit(X_train,y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [27]:
y_eval_pred = model.predict(X_eval)

**Ещё раз оцениваем модель**

In [28]:
mean_squared_error(y_eval,y_eval_pred)

2.383783075056986

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [29]:
y_final_test_pred = model.predict(X_test)

In [30]:
mean_squared_error(y_test,y_final_test_pred)

2.254260083800517

----
----
----
## Кросс-валидация с помощью cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [44]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабирование данных (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [45]:
model = Ridge(alpha=100)

In [46]:
from sklearn.model_selection import cross_val_score

In [48]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [49]:
scores

array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
        -8.38562723])

In [50]:
# Среднее значение MSE scores (делаем это значение положительным)
abs(scores.mean())

8.215396464543607

**Уточняем модель на основе метрик**

In [51]:
model = Ridge(alpha=1)

In [52]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [53]:
# Среднее значение MSE scores (делаем это значение положительным)
abs(scores.mean())

3.344839296530695

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [55]:
# Сначала нужно обучить модель!
model.fit(X_train,y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [56]:
y_final_test_pred = model.predict(X_test)

In [57]:
mean_squared_error(y_test,y_final_test_pred)

2.319021579428752

----
----
----

# Кросс-валидация с помощью cross_validate

Функция cross_validate отличается от cross_val_score двумя аспектами:

эта функция позволяет использовать для оценки несколько метрик;

она возвращает не только оценку на тестовом наборе (test score), но и словарь с замерами времени обучения и скоринга, а также - опционально - оценки на обучающем наборе и объекты estimator.

В случае одной метрики для оценки, когда параметр scoring является строкой string, вызываемым объектом callable или значением None, ключи словаря будут следующими:
        
        - ['test_score', 'fit_time', 'score_time']

А в случае нескольких метрик для оценки, возвращаемый словарь будет содержать следующие ключи:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score по умолчанию принимает значение False, чтобы сэкономить вычислительные ресурсы. Чтобы посчитать оценки на обучающем наборе, достаточно установить этот параметр в значение True.

In [62]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Делаем разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [63]:
model = Ridge(alpha=100)

In [64]:
from sklearn.model_selection import cross_validate

In [72]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [73]:
scores

{'fit_time': array([0.00102687, 0.00088882, 0.00099993, 0.00099945, 0.        ]),
 'score_time': array([0.00108409, 0.        , 0.        , 0.00064516, 0.00086308]),
 'test_neg_mean_absolute_error': array([-2.31243044, -1.74653361, -2.56211701, -2.01873159, -2.27951906]),
 'test_neg_mean_squared_error': array([ -9.32552967,  -4.9449624 , -11.39665242,  -7.0242106 ,
         -8.38562723]),
 'test_max_error': array([ -6.44988486,  -5.58926073, -10.33914027,  -6.61950405,
         -7.75578515])}

In [74]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.001027,0.001084,-2.312430,-9.325530,-6.449885
1,0.000889,0.000000,-1.746534,-4.944962,-5.589261
2,0.001000,0.000000,-2.562117,-11.396652,-10.339140
3,0.000999,0.000645,-2.018732,-7.024211,-6.619504
4,0.000000,0.000863,-2.279519,-8.385627,-7.755785


In [75]:
pd.DataFrame(scores).mean()

fit_time                        0.000783
score_time                      0.000518
test_neg_mean_absolute_error   -2.183866
test_neg_mean_squared_error    -8.215396
test_max_error                 -7.350715
dtype: float64

**Уточняем модель на основе метрик**

In [76]:
model = Ridge(alpha=1)

In [77]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [78]:
pd.DataFrame(scores).mean()

fit_time                        0.000901
score_time                      0.000200
test_neg_mean_absolute_error   -1.319685
test_neg_mean_squared_error    -3.344839
test_max_error                 -5.161145
dtype: float64

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [79]:
# Сначала нужно обучить модель!
model.fit(X_train,y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [80]:
y_final_test_pred = model.predict(X_test)

In [81]:
mean_squared_error(y_test,y_final_test_pred)

2.319021579428752

----
----